# 火币数据收集程序(Huobi Candle Data Collecting)

- 数据源:**火币（Huobi）**[https://api.huobi.io]
- 主要币种：10 coins
- 存储对象：mysql
- pip install mysql-connector


-  **获取蜡烛图(Candle Collecting)**

## 创建数据库(Create Database)：

CREATE DATABASE IF NOT EXISTS Huobi_Ex DEFAULT CHARSET utf8 COLLATE utf8_general_ci;

## 创建数据表(Create Tables)：

下面会通过这个模板创建对应的表
CREATE TABLE kline_%(symbol)s_%(period)s 
    (`amount` decimal(30,18) , `open` decimal(30,18), `close` decimal(30,18), `high` decimal(30,18), `id` bigint,
    `count` decimal(30,18), `low` decimal(30,18), `vol` decimal(30,18),
    constraint primary key(id));

In [ ]:
#from gevent import monkey; monkey.patch_all()
import time
import requests
import json
import json
import mysql.connector as db
import random
#import gevent
import os
from IPython.display import clear_output as clear
import datetime

In [ ]:
db_config = {
    "database": "Huobi_Ex",
    "user": "root",
    "password": "root",
    "host": "localhost",
    "port": 3306,
}

# 连接数据库获取游标，可以设置返回数据的格式，元组，命令元组，字典等...
connect = db.Connect(**db_config)

In [ ]:
periods = ['1min', '5min', '15min', '30min', '60min', '4hour', '1day', '1week', '1mon', '1year']

symbols = ['btcusdt', 'ethusdt', 'htusdt', 'xrpusdt', 'ltcusdt', 'bchusdt', 'eosusdt', 'etcusdt', 'bsvusdt', 'linkusdt', \
          'ethbtc', 'htbtc', 'xrpbtc', 'ltcbtc', 'bchbtc', 'eosbtc', 'etcbtc', 'bsvbtc', 'linkbtc', ]

cpair = len(symbols)

In [ ]:
def getCreateTableScript():
    print("# create table script");
    for period in periods: 
        for symbol in symbols:
            print("CREATE TABLE kline_%(symbol)s_%(period)s("%{"symbol":symbol, "period":period})
            print("  `amount` decimal(30,18) , `open` decimal(30,18), `close` decimal(30,18), `high` decimal(30,18), `id` bigint,")
            print("  `count` decimal(30,18), `low` decimal(30,18), `vol` decimal(30,18),")
            print("  constraint primary key(id)")
            print(");");

In [ ]:
def OnMarketKline(symbol='btcusdt', period='1min', size =2000, isFile = False):
    i = 0
    j = 0
    if size>2000 :
        size = 2000
    elif size < 200 :
        size = 200
    #&AccessKeyId=%(AccessKeyId)s&Timestamp=%(Timestamp)s
    url="https://api.huobi.pro/market/history/kline?period=%(period)s&size=%(size)s&symbol=%(symbol)s" \
        %{'symbol':symbol, 'period':period, 'size':size, 'AccessKeyId':'ae957d1d-e1d269ee-18b8966a-bewr5drtmh','Timestamp':time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())}
    
    #print(url)
    #{'Content-Type': 'application/json'}, 

    try:
        if isFile == False:
            if isProxy == False:
                response = requests.get(url, {'Content-Type': 'application/json'}, verify=True, timeout=600) #,verify=False
            else:
                proxDict = {"http":"127.0.0.1:1881",
                            "https":"127.0.0.1:1881",
                           }
                response = requests.get(url, {'Content-Type': 'application/json'}, verify=True, timeout=600, proxies=proxDict) #,verify=False
            result = json.loads(response.text)
        else :
            f = open('./resp/'+symbol+'_'+period+'.json')
            lines = f.read()
            f.close()
            result = json.loads(lines)
            
        if  result["status"] == 'ok' and result["ch"] != False :
            ch = result["ch"].split(".")
            symbol=ch[1]
            period=ch[3]

        if result["status"] == 'ok':
            cursor = connect.cursor()
            #'amount","open": ,"close": ,"high": ,"id": ,"count": ,"low": ,"vol"
            sqli = "replace into `kline_%(symbol)s_%(period)s` (`amount`, `open`, `close`, `high`, `id`, `count`, `low`, `vol`) \
                     select %(amount)s, %(open)s, %(close)s, %(high)s, %(id)s, %(count)s, %(low)s, %(vol)s where \
                     not exists(select id from `kline_%(symbol)s_%(period)s` where id = %(id)s and amount = %(amount)s) "

            for item in result["data"] :
                item.update({"symbol":symbol,"period":period})
                sqlcmd = sqli%(item)
                #print(sqlcmd)
                cursor.execute(sqlcmd)

                if cursor.rowcount == 2 :
                    j += 1
                else :
                    i+=cursor.rowcount
                
                #print(cursor.rowcount)
            i += j//2
            connect.commit()
            print("keypair:%s :: period:%s,  rows:%s, effect rows:%s, update rows:%s || %s"%(symbol.rjust(10,' '), period.rjust(8,' '), str(len(result["data"])).rjust(5,' '), str(i).rjust(5,' '), str(j).rjust(5,' '), time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())))
            cursor.close()
        else :
            print("status:%s, ch:%s, data:%s"%(result["status"],result["ch"],len(result["data"])))
        
        if i == 0 and len(result["data"]) > 0:
            i = -1
        return i
    except Exception as e:
        if hasattr(e,"reason"):
            print("失败原因:",e.reason)
        if hasattr(e,"code"):
            print("错误代码:",e.code)
            print("返回内容:",e.read())
        else:
            print(e)
    
    return i

In [ ]:
def checkingForCompleteScript(symbol='btcusdt'):
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/60 as calitems, count(*) as realitems from `kline_"+ symbol +"_1min`;")
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/(60*5) as calitems, count(*) as realitems from `kline_"+ symbol +"_5min`;")
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/(60*15) as calitems, count(*) as realitems from `kline_"+ symbol +"_15min`;")
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/(60*30) as calitems, count(*) as realitems from `kline_"+ symbol +"_30min`;")
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/(60*60) as calitems, count(*) as realitems from `kline_"+ symbol +"_60min`;")
    print("select max(id) as maxid, min(id) as minid, (max(id) - min(id))/(60*240) as calitems, count(*) as realitems from `kline_"+ symbol +"_4hour`;")

In [ ]:
def getoutputscript(symbol='btcusdt', period='1min', outpath = 'C:/mysql_data/output/'):
    dateflag17 = datetime.date(2017, 1, 1)
    flag17 = str(int(time.mktime(dateflag17.timetuple())))
    
    dateflag18 = datetime.date(2018, 1, 1)
    flag18 = str(int(time.mktime(dateflag18.timetuple())))
    
    dateflag19 = datetime.date(2019, 1, 1)
    flag19 = str(int(time.mktime(dateflag19.timetuple())))
    
    dateflag20 = datetime.date(2020, 1, 1)
    flag20 = str(int(time.mktime(dateflag20.timetuple())))
    
    dateflag20_cMonth = datetime.date(2020, 5, 1)
    flag20_cMonth = str(int(time.mktime(dateflag20_cMonth.timetuple())))
    
    print("select * from (SELECT 'id', 'open', 'high', 'low', 'close', 'vol', 'count', 'amount' union")
    print("SELECT `id`, `open`, `high`, `low`, `close`, `vol`, `count`, `amount` ")
    print("FROM kline_" + symbol + "_"+ period +" WHERE id >=  "+ flag17 +" and id < " + flag18 + ") ohlcv INTO OUTFILE '"+ outpath + symbol + "_"+ period +"_2017.csv'  FIELDS TERMINATED BY ',';")
    print("")
    print("select * from (SELECT 'id', 'open', 'high', 'low', 'close', 'vol', 'count', 'amount' union")
    print("SELECT `id`, `open`, `high`, `low`, `close`, `vol`, `count`, `amount` ")
    print("FROM kline_" + symbol + "_"+ period +" WHERE id >=  "+ flag18 +" and id < " + flag19 + ") ohlcv INTO OUTFILE '"+ outpath + symbol + "_"+ period +"_2018.csv'  FIELDS TERMINATED BY ',';")
    print("")
    print("select * from (SELECT 'id', 'open', 'high', 'low', 'close', 'vol', 'count', 'amount' union")
    print("SELECT `id`, `open`, `high`, `low`, `close`, `vol`, `count`, `amount` ")
    print("FROM kline_" + symbol + "_"+ period +" WHERE id >=  "+ flag19 +" and id < " + flag20 + ") ohlcv INTO OUTFILE '"+ outpath + symbol + "_"+ period +"_2019.csv'  FIELDS TERMINATED BY ',';")
    print("")
    print("select * from (SELECT 'id', 'open', 'high', 'low', 'close', 'vol', 'count', 'amount' union")
    print("SELECT `id`, `open`, `high`, `low`, `close`, `vol`, `count`, `amount` ")
    print("FROM kline_" + symbol + "_"+ period +" WHERE id >=  "+ flag20 +" and id < " + flag20_cMonth + ") ohlcv INTO OUTFILE '"+ outpath + symbol + "_"+ period +"_2020_1-4.csv'  FIELDS TERMINATED BY ',';")
    print("")

In [ ]:
getCreateTableScript()

In [ ]:
getoutputscript(symbols[0])

In [ ]:
checkingForCompleteScript(symbols[0])

In [ ]:
loopc = 0
loopx = 0
maxloop = 100
nextloop = True
baseticks = 10
isFile = False
isProxy = True

while nextloop:
    # loopx = 0
    ticks = random.randint(baseticks+5,baseticks+15); #间隔采集频率 10-30秒
    
    if loopx == 0:
        os.system('cls' if os.name == 'nt' else 'clear')
        clear()
    
    size = 2000
    hashperiod = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    periodk = periods[(loopx//cpair)%10]
    periodc = hashperiod[(loopx//cpair)%10]
    
    if loopc > 0 :
        size = 200
        hashperiod = [0, 1, 0, 2, 0, 3, 0, 4, 0, 5, 0, 1, 0, 6, 0, 7, 0, 8, 0, 9]
        periodk = periods[hashperiod[(loopx//cpair)%20]]
        periodc = hashperiod[(loopx//cpair)%20]
        baseticks = 15
        
    symbolk = loopx%cpair
    
    if loopc>0 and hashperiod[(loopx//cpair)%20] < 2:
        size = 200 
    elif loopc>0 :
        size = 10
    
    #print("%s---OnMarketKline(%s, %s, %s)"%(str(loopx).rjust(5,'0'), symbols[symbolk], periodk, size))
    #print("%s"%(str(loopx).rjust(5,'0')))
    
    size = OnMarketKline(symbols[symbolk], periodk, size, isFile)
    if size == 0:
        print("try again:")
        size = 2000;
        size = OnMarketKline(symbols[symbolk], periodk, size, isFile)
    
    loopx = loopx + 1
    
    if loopx == cpair * 10 and  loopc == 0 :
        loopc = 1
        loopx = 0
    elif loopx == cpair * 20 and  loopc > 0 :
        loopc += 1
        loopx = 0

    time.sleep(ticks) 
    
    if loopc >= maxloop:
        nextloop = False
 
#close connect
connect.close()
print("Exit")